In [1]:
import ROOT
import numpy as np

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x79dfec0


In [2]:
%jsroot on

In [3]:
# FIXME: don't touch! the fortran code has a global state :(
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.gSystem.Load("OO/diehl/libObservable.so")

0

In [5]:
ROOT.gInterpreter.Declare("#include \"OO/diehl/OOTools_old.h\"")

True

In [6]:
# use same masses as in whizard!
mW = 80.419
mZ = 91.1882

# ROOT.OOTools.init(250, mW, mZ)
ROOT.OOTools.init(190, mW, mZ)

In [7]:
ROOT.ildStyle.SetOptStat(1)
# ROOT.ildStyle.SetOptStat(0)

In [8]:
%%cpp
using namespace ROOT::VecOps;

In [9]:
n_points = 5000000
# n_points = 1000000

df = ROOT.RDataFrame(n_points)

In [10]:
df = df.Define("co", "gRandom->Uniform(-1., 1.)")
df = df.Define("co1", "gRandom->Uniform(-1., 1.)")
df = df.Define("co2", "gRandom->Uniform(-1., 1.)")
df = df.Define("ph1", "gRandom->Uniform(-1.*ROOT::Math::Pi(), 1.*ROOT::Math::Pi())")
df = df.Define("ph2", "gRandom->Uniform(-1.*ROOT::Math::Pi(), 1.*ROOT::Math::Pi())")

In [11]:
g = 0.002

df = df.Define("S_0", "OOTools::proba(co, co1, co2, ph1, ph2, 0, true, 0)")
df = df.Define("S_1", "OOTools::proba(co, co1, co2, ph1, ph2, 1, false, 1)")
df = df.Define("w_proba", "S_0")

df = df.Define("d_sigma", f"S_0 + {g} * S_1")
df = df.Define("S", f"S_0 + {g} * S_1")
df = df.Vary("w_proba", "RVec({d_sigma})", ["fz1"], "aTGC")
df = df.Define("O", "S_1 / S_0")

# as we are sampling phase space uniformly we can also get rid of weighting by S and just use
# these three quantities to calculate g as
# g = 1/<a> * (<b> - <c>)
# with a = S_1^2 / S_0, b = S_1 * S / S_0, c = S_1
df = df.Define("a", "S_1 * S_1 / S_0")
df = df.Define("b", "S_1 * S / S_0")
df = df.Define("c", "S_1")

In [12]:
m_a = df.Mean("a")
m_b = df.Mean("b")
m_c = df.Mean("c")

In [13]:
pi = ROOT.Math.Pi()

# or just fill 1D histograms with weight proba??
# p_co = df.Profile1D(("", ";co", 25, -1., 1.), "co", "proba")
# p_co1 = df.Profile1D(("", ";co1", 25, -1., 1.), "co1", "proba")
# p_co2 = df.Profile1D(("", ";co2", 25, -1., 1.), "co2", "proba")
# p_ph1 = df.Profile1D(("", ";ph1", 25, -pi, pi), "ph1", "proba")
# p_ph2 = df.Profile1D(("", ";ph2", 25, -pi, pi), "ph2", "proba")
h_co  = df.Histo1D(("", ";co", 25, -1., 1.), "co", "w_proba")
h_co1 = df.Histo1D(("", ";co1", 25, -1., 1.), "co1", "w_proba")
h_co2 = df.Histo1D(("", ";co2", 25, -1., 1.), "co2", "w_proba")
h_ph1 = df.Histo1D(("", ";ph1", 25, -pi, pi), "ph1", "w_proba")
h_ph2 = df.Histo1D(("", ";ph2", 25, -pi, pi), "ph2", "w_proba")

v_co = ROOT.RDF.Experimental.VariationsFor(h_co)
v_co1 = ROOT.RDF.Experimental.VariationsFor(h_co1)
v_co2 = ROOT.RDF.Experimental.VariationsFor(h_co2)
v_ph1 = ROOT.RDF.Experimental.VariationsFor(h_ph1)
v_ph2 = ROOT.RDF.Experimental.VariationsFor(h_ph2)

h_weight = df.Histo1D("w_proba")
h_O = df.Histo1D(("", ";O = S_1 / S_0", 100, -1.5, 1.5), "O", "w_proba")
v_O = ROOT.RDF.Experimental.VariationsFor(h_O)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xba41c80


In [14]:
a = m_a.GetValue()
b = m_b.GetValue()
c = m_c.GetValue()

gamma = 1/a * (b - c)
print(gamma)

0.0020000000000000035


In [15]:
c_O = ROOT.TCanvas()
h_O.Draw()
v_O["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_O["aTGC:fz1"].Draw("same)")
c_O.Draw()

c_co = ROOT.TCanvas()
h_co.SetMinimum(0)
h_co.Draw()
v_co["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_co["aTGC:fz1"].Draw("same)")
c_co.Draw()

c_co1 = ROOT.TCanvas()
h_co1.SetMinimum(0)
h_co1.Draw()
v_co1["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_co1["aTGC:fz1"].Draw("same)")
c_co1.Draw()

c_ph1 = ROOT.TCanvas()
h_ph1.SetMinimum(0)
h_ph1.Draw()
v_ph1["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_ph1["aTGC:fz1"].Draw("same)")
c_ph1.Draw()

c_co2 = ROOT.TCanvas()
h_co2.SetMinimum(0)
h_co2.Draw()
v_co2["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_co2["aTGC:fz1"].Draw("same)")
c_co2.Draw()

c_ph2 = ROOT.TCanvas()
h_ph2.SetMinimum(0)
h_ph2.Draw()
v_ph2["aTGC:fz1"].SetMarkerColor(ROOT.kRed)
v_ph2["aTGC:fz1"].Draw("same)")
c_ph2.Draw()

c_weight = ROOT.TCanvas()
h_weight.SetMinimum(0)
h_weight.Draw()
c_weight.Draw()